In [1]:
%cd ..

/Users/heilman/galvanize/recipe-vectorizer


In [2]:
from src.functions import *
from src.modeling_functions import *
import numpy as np
import networkx as nx
import pandas as pd
from collections import Counter
from string import *
from sklearn.metrics.pairwise import cosine_similarity
from sys import argv, exit
import pymongo
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm

In [3]:
mc = pymongo.MongoClient()
db = mc['allrecipes']
recipes_coll = db['recipes']

In [6]:
recipes, ratings = find_recipes_matching_search(recipes_coll, 'chili')

In [7]:
len(recipes)

296

In [9]:
units_by_ing = get_common_units_by_ingredient(recipes)

In [17]:
units_by_ing['onion'].most_common(1)[0][0]

'each'

In [ ]:
conversion_dict

In [76]:
def convert_qty_to_common_units(ingredient,
                                norm_qty,
                                units_by_ing,
                                conversion_dict=conversion_dict):
    desired_units = units_by_ing[ingredient].most_common(1)[0][0]
    volume_based_units = ['teaspoon','tablespoon','cup','pint','quart','gallon','ounce','fluid ounce','milliliter',
                          'pinch','dash']
    if desired_units in volume_based_units:
        return decode_volume_units(norm_qty)
    elif desired_units == 'pound':
            return '{0:1.2} pounds'.format(norm_qty / 16)
    elif desired_units == 'each':
        ingredient_conversions = conversion_dict[desired_units]
        try:
            return str(round(norm_qty / ingredient_conversions[ingredient]))
        except:
            return str(round(norm_qty / ingredient_conversions['other']))
    else:
        try:
            ingredient_conversions = conversion_dict[desired_units]
            try:
                return '{0} {1}'.format(round(norm_qty/ingredient_conversions[ingredient]), desired_units)
            except:
                return '{0} {1}'.format(round(norm_qty/ingredient_conversions['other']), desired_units)
        except:
            return '{0} {1}'.format(norm_qty, desired_units)

In [57]:
convert_qty_to_common_units('bell pepper', 4, units_by_ing)

'1'

In [43]:
def decode_volume_units(norm_qty):
    if norm_qty < 1/24 * 1.5:
        return '1/4 teaspoon'
    elif norm_qty < 1/12 * 1.5:
        return '1/2 teaspoon'
    elif norm_qty < 1/6 * 1.25:
        return '1 teaspoon'
    elif norm_qty < 1/6 * 1.75:
        return '1 1/2 teaspoons'
    elif norm_qty < 1/6 * 2.25:
        return '2 teaspoons'
    elif norm_qty < 1/6 * 2.75:
        return '2 1/2 teaspoons'
    elif norm_qty < 1/2 * 1.25:
        return '1 tablespoon'
    elif norm_qty < 1/2 * 1.75:
        return '1 1/2 tablespoons'
    elif norm_qty < 1.25:
        return '2 tablespoons'
    elif norm_qty < 1.75:
        return '3 tablespoons'
    elif norm_qty < 2.3:
        return '1/4 cup'
    elif norm_qty < 3:
        return '1/3 cup'
    elif norm_qty < 5:
        return '1/2 cup'
    elif norm_qty < 7:
        return '3/4 cup'
    elif norm_qty < 9:
        return '1 cup'
    elif norm_qty < 11:
        return '1 1/4 cups'
    elif norm_qty < 14:
        return '1 1/2 cups'
    elif norm_qty < 18:
        return '2 cups'
    elif norm_qty < 22:
        return '2 1/2 cups'
    elif norm_qty < 28:
        return '3 cups'
    elif norm_qty < 36:
        return '1 quart'
    else:
        return '{0:1.2} quarts'.format(norm_qty / 32)

In [59]:
#recipes[0]

{'_id': ObjectId('5cae848ec6d0b07b2dbbe5cb'),
 'id': 78299,
 'name': 'Boilermaker Tailgate Chili',
 'href': 'https://www.allrecipes.com/recipe/78299/boilermaker-tailgate-chili/',
 'category': {'lvl_1': 'Soups, Stews and Chili',
  'lvl_2': 'Chili',
  'lvl_3': 'Pork Chili'},
 'rating_info': {'rating': 4.81840705871582,
  'made_by': 12000,
  'reviews': 4000},
 'submitter_info': {'id': 591623,
  'name': 'MIGHTYPURDUE22',
  'followers': 41,
  'href': 'https://www.allrecipes.com/cook/591623/'},
 'ingredients': {'beef chuck': {'quantity': 2.0,
   'units': 'pound',
   'normalized_qty': 32.0,
   'category': 'other'},
  'italian sausage': {'quantity': 1.0,
   'units': 'pound',
   'normalized_qty': 16.0,
   'category': 'protein'},
  'chili beans': {'quantity': 45.0,
   'units': 'ounce',
   'normalized_qty': 45.0,
   'category': 'vegetable'},
  'chili beans in spicy sauce': {'quantity': 15.0,
   'units': 'ounce',
   'normalized_qty': 15.0,
   'category': 'other'},
  'tomato': {'quantity': 56.0,
  

In [61]:
df = create_dataframe(recipes)
X = df.values
cosine_sims = cosine_similarity(X)

G = create_graph(cosine_sims, threshold=0.7)
remove_isolates(G, 5)

In [62]:
recipe_results = generate_recipes(G, df)

In [78]:
for i, qty in enumerate(recipe_results[0]):
    ing = recipe_results[0].index[i]
    print(convert_qty_to_common_units(ing, qty*10, units_by_ing), ing)

1 onion
2 tablespoons chili powder
2 1/2 cups tomato
1 1/2 teaspoons cumin
2 clove garlic
1.0 pounds beef
1 1/2 cups kidney beans
1 bell pepper
1/2 teaspoon salt
1 teaspoon oregano
3/4 cup tomato sauce
1/4 teaspoon black pepper
1/2 cup water
1 1/2 teaspoons olive oil
1/4 teaspoon cayenne pepper
3/4 cup black beans
2 tablespoons tomato paste
1 1/2 teaspoons vegetable oil
1/2 cup chicken broth
2 tablespoons green chile


In [64]:
type(recipe_results[0])

pandas.core.series.Series